In [31]:
import pandas as pd
import numpy as np

In [32]:
# Load the bradycardia BPM dataset (to be rescaled for tachycardia)
bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/CSV_DATA/BPM_8hr_5min.csv"
df = pd.read_csv(bpm_file, header=None)  # Assuming no headers in file

In [33]:
# Define original adult BPM ranges and the new neonatal tachycardia range
adult_normal_range = (60, 100)
adult_tachycardia_range = (70, 180)  # Lowered minimum to capture more tachycardia cases
neonatal_normal_range = (100, 160)
neonatal_tachycardia_range = (160, 250)  # Increased upper bound to allow higher BPM

In [34]:
# Function to rescale values
def rescale_bpm(data, old_min, old_max, new_min, new_max):
    return ((data - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

In [35]:
# Apply rescaling
df_tachycardia = df.copy()

# Scale normal BPM values (60-100) -> Neonatal normal range (100-160)
df_tachycardia[df < 100] = rescale_bpm(df[df < 100], *adult_normal_range, *neonatal_normal_range)

# Scale higher BPM values (≥70) -> Neonatal tachycardia range (160-250)
df_tachycardia[df >= 70] = rescale_bpm(df[df >= 70], *adult_tachycardia_range, *neonatal_tachycardia_range)

In [36]:
# **Artificially increase tachycardia cases**
# Select 50% of normal BPM cases and force them into tachycardia
normal_indices = df_tachycardia[df_tachycardia < 160].stack().index
selected_indices = np.random.choice(len(normal_indices), int(len(normal_indices) * 0.5), replace=False)

In [37]:
for idx in selected_indices:
    row, col = normal_indices[idx]
    df_tachycardia.iloc[row, col] = np.random.uniform(160, 230)  # Force into tachycardia range

In [38]:
# Introduce additional random spikes to tachycardia
tachycardia_mask = df_tachycardia > 160
df_tachycardia[tachycardia_mask] += np.random.uniform(10, 30, size=df_tachycardia[tachycardia_mask].shape)

In [39]:
# Save the rescaled neonatal tachycardia dataset
tachycardia_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv"
df_tachycardia.to_csv(tachycardia_bpm_file, index=False)

print(f"Neonatal Tachycardia BPM data saved: {tachycardia_bpm_file}")

Neonatal Tachycardia BPM data saved: /home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv


In [40]:
import pandas as pd

# Load the neonatal BPM dataset for tachycardia
neonatal_bpm_file = "/home/Gurshan.R/Documents/GitHub/SYSC4907_Capstone/GAN_Tachycardia/Scaled_CSV_DATA/BPM_Tachycardia_8hr_5min_Neonatal.csv"
df_neonatal = pd.read_csv(neonatal_bpm_file, header=None)

# Remove the first row if it contains index-like values
if (df_neonatal.iloc[0] == range(96)).all():  # Check if first row is just index values
    df_neonatal = df_neonatal.iloc[1:].reset_index(drop=True)  # Remove it

# Convert to numeric to ensure proper processing
df_neonatal = df_neonatal.apply(pd.to_numeric, errors="coerce")

# Define neonatal tachycardia thresholds
tachycardia_threshold = 160
severe_tachycardia_threshold = 230

# Identify tachycardia instances (BPM > 160)
tachycardia_instances = df_neonatal[df_neonatal > tachycardia_threshold].stack().reset_index()
tachycardia_instances.columns = ["Row", "Column", "BPM"]

# Identify severe tachycardia instances (BPM > 200)
severe_tachy_instances = tachycardia_instances[tachycardia_instances["BPM"] > severe_tachycardia_threshold]

# Print the detected tachycardia instances
print("\nTachycardia Instances (BPM > 160):")
print(tachycardia_instances)

print("\nSevere Tachycardia Instances (BPM > 230):")
print(severe_tachy_instances)



Tachycardia Instances (BPM > 160):
     Row  Column         BPM
0      0       0  174.090720
1      0       1  179.262172
2      0       2  187.323823
3      0       3  181.719542
4      0       4  190.133405
..   ...     ...         ...
894    9      89  194.214730
895    9      90  193.226792
896    9      91  188.610369
897    9      94  185.824385
898    9      95  199.535901

[899 rows x 3 columns]

Severe Tachycardia Instances (BPM > 230):
     Row  Column         BPM
92     0      93  249.292086
187    1      93  247.596973
286    3       0  230.639862
288    3       3  245.889865
296    3      13  248.612717
366    3      89  235.960289
372    4       1  230.270913
445    4      78  233.903581
473    5      12  248.688302
515    5      58  242.292618
523    5      67  238.312492
536    5      81  254.398746
846    9      29  243.380629
848    9      32  240.581303
858    9      42  236.945619
868    9      54  243.064245
891    9      84  237.149102
